In [35]:
import numpy as np
import pandas as pd
import math
pd.set_option('display.max_columns', 100)
pd.options.display.float_format = '{:.2f}'.format

# Outline

###  Part A. Affordable  Rentals
 1. Bring in 2017 PUMS data
 2. Assign county_id to each puma
 3. Bring in and clean, reformat county AMIs data
 4. Add variables for cost limits for ELI, VLI, LI, MI, and HI by bedroom count for each county
 5. Add count dummies for each unit based on income and unit type
 

###  Part B. Affordable Owning
 1. Calculate total monthly housing cost by adding monthly first mortgage, second/junior mortgage, and condo fees
 2. Add count dummies for each unit based on income and unit type

### Part C. Weighting Sample to get estimates
 1. Multiply each dummy by the the household weight variable
 2. Calculte standard errors for each estimate
 
### Part D. Aggregating at PUMA Level
 1. Add dummy variables for vacant "for-rent," vacant "for-sale," and recently moved in and create second dataset
 2. Aggregate at PUMA level for each dataset and export

## Part A1. Bring in 2017 PUMS data

In [36]:
variable_types = {"BDSP":"float","RMSP":"str","MV":"float","WGTP":"float","TAXP":"float"}
columns = ["PUMA","RT","BDSP","BLD","RNTP","MRGP","SMP","CONP","TEN","VACS","TAXP","VALP","GRPIP","GRNTP","HINCP","MV","WGTP"]
pums_df=pd.read_csv("2017_pums.csv", delimiter=",",usecols=columns,dtype=variable_types)
pums_df

,RT,PUMA,WGTP,BDSP,BLD,CONP,MRGP,RNTP,SMP,TEN,VACS,VALP,GRNTP,GRPIP,HINCP,MV,TAXP
0,H,6703,59.00,2.00,9.00,nan,nan,940.00,nan,3.00,nan,nan,990.00,25.00,47000.00,3.00,nan
1,H,110,74.00,4.00,2.00,0.00,2600.00,nan,nan,1.00,nan,780000.00,nan,nan,130500.00,1.00,67.00
2,H,7901,111.00,2.00,6.00,nan,nan,nan,nan,nan,5.00,nan,nan,nan,nan,nan,nan
3,H,1906,63.00,3.00,2.00,0.00,nan,nan,nan,2.00,nan,175000.00,nan,nan,13530.00,7.00,14.00
4,H,6710,75.00,5.00,2.00,0.00,nan,nan,nan,2.00,nan,500000.00,nan,nan,104000.00,4.00,48.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157729,H,6710,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
157730,H,2901,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
157731,H,3900,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
157732,H,2500,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [37]:
pums_df.rename(columns={"RT":"record_type","BDSP":"number_bedrooms",
    "BLD":"units_in_structure",
    "RNTP":"monthly_rent",
    "MRGP":"first_mortgage",
    "SMP":"second_mortgage",
    "CONP":"condo_fee",
    "TEN":"tenure",
    "VACS":"vacancy_status",
    "VALP":"property_value",
    "GRPIP":"gross_rent_pct_of_income",
    "GRNTP":"gross_rent",
    "HINCP":"hh_income",
    "TAXP":"prop_tax",
    "WGTP":"weight",
    "MV":"moved_in"}, inplace=True)
pums_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax
0,H,6703,59.00,2.00,9.00,nan,nan,940.00,nan,3.00,nan,nan,990.00,25.00,47000.00,3.00,nan
1,H,110,74.00,4.00,2.00,0.00,2600.00,nan,nan,1.00,nan,780000.00,nan,nan,130500.00,1.00,67.00
2,H,7901,111.00,2.00,6.00,nan,nan,nan,nan,nan,5.00,nan,nan,nan,nan,nan,nan
3,H,1906,63.00,3.00,2.00,0.00,nan,nan,nan,2.00,nan,175000.00,nan,nan,13530.00,7.00,14.00
4,H,6710,75.00,5.00,2.00,0.00,nan,nan,nan,2.00,nan,500000.00,nan,nan,104000.00,4.00,48.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157729,H,6710,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
157730,H,2901,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
157731,H,3900,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
157732,H,2500,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


## Part A2. Assign county id to each puma

In [38]:
# load in crosswalk file
crosswalk_df=pd.read_csv("C:/Users/Terner GSR/Box/Cost of Doing Nothing/Climate Impacts/Data/PUMS/PUMA_County_Crosswalk_v2.csv", delimiter=",")
crosswalk_df

,PUMA,county1,county2,county3,county4,county5,county6,county7
0,101,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN
1,102,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN
2,103,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN
3,104,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN
4,105,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
260,11103,Ventura CA,NaN,NaN,NaN,NaN,NaN,NaN
261,11104,Ventura CA,NaN,NaN,NaN,NaN,NaN,NaN
262,11105,Ventura CA,NaN,NaN,NaN,NaN,NaN,NaN
263,11106,Ventura CA,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
# add county name column to puma file
puma_county_df=pums_df.merge(crosswalk_df, how='left', left_on = "PUMA",right_on = "PUMA")
puma_county_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7
0,H,6703,59.00,2.00,9.00,nan,nan,940.00,nan,3.00,nan,nan,990.00,25.00,47000.00,3.00,nan,Sacramento CA,NaN,NaN,NaN,NaN,NaN,NaN
1,H,110,74.00,4.00,2.00,0.00,2600.00,nan,nan,1.00,nan,780000.00,nan,nan,130500.00,1.00,67.00,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN
2,H,7901,111.00,2.00,6.00,nan,nan,nan,nan,nan,5.00,nan,nan,nan,nan,nan,nan,San Luis Obispo CA,NaN,NaN,NaN,NaN,NaN,NaN
3,H,1906,63.00,3.00,2.00,0.00,nan,nan,nan,2.00,nan,175000.00,nan,nan,13530.00,7.00,14.00,Fresno CA,NaN,NaN,NaN,NaN,NaN,NaN
4,H,6710,75.00,5.00,2.00,0.00,nan,nan,nan,2.00,nan,500000.00,nan,nan,104000.00,4.00,48.00,Sacramento CA,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157729,H,6710,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Sacramento CA,NaN,NaN,NaN,NaN,NaN,NaN
157730,H,2901,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Kern CA,NaN,NaN,NaN,NaN,NaN,NaN
157731,H,3900,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Madera CA,NaN,NaN,NaN,NaN,NaN,NaN
157732,H,2500,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Imperial CA,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
# remove " CA" from end of each county name
puma_county_df['County'] = puma_county_df['county1'].str.replace(r' CA', '')
puma_county_df['county2'] = puma_county_df['county2'].str.replace(r' CA', '')
puma_county_df['county3'] = puma_county_df['county3'].str.replace(r' CA', '')
puma_county_df['county4'] = puma_county_df['county4'].str.replace(r' CA', '')
puma_county_df['county5'] = puma_county_df['county5'].str.replace(r' CA', '')
puma_county_df['county6'] = puma_county_df['county6'].str.replace(r' CA', '')
puma_county_df['county7'] = puma_county_df['county7'].str.replace(r' CA', '')
puma_county_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County
0,H,6703,59.00,2.00,9.00,nan,nan,940.00,nan,3.00,nan,nan,990.00,25.00,47000.00,3.00,nan,Sacramento CA,NaN,NaN,NaN,NaN,NaN,NaN,Sacramento
1,H,110,74.00,4.00,2.00,0.00,2600.00,nan,nan,1.00,nan,780000.00,nan,nan,130500.00,1.00,67.00,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN,Alameda
2,H,7901,111.00,2.00,6.00,nan,nan,nan,nan,nan,5.00,nan,nan,nan,nan,nan,nan,San Luis Obispo CA,NaN,NaN,NaN,NaN,NaN,NaN,San Luis Obispo
3,H,1906,63.00,3.00,2.00,0.00,nan,nan,nan,2.00,nan,175000.00,nan,nan,13530.00,7.00,14.00,Fresno CA,NaN,NaN,NaN,NaN,NaN,NaN,Fresno
4,H,6710,75.00,5.00,2.00,0.00,nan,nan,nan,2.00,nan,500000.00,nan,nan,104000.00,4.00,48.00,Sacramento CA,NaN,NaN,NaN,NaN,NaN,NaN,Sacramento
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157729,H,6710,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Sacramento CA,NaN,NaN,NaN,NaN,NaN,NaN,Sacramento
157730,H,2901,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Kern CA,NaN,NaN,NaN,NaN,NaN,NaN,Kern
157731,H,3900,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Madera CA,NaN,NaN,NaN,NaN,NaN,NaN,Madera
157732,H,2500,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Imperial CA,NaN,NaN,NaN,NaN,NaN,NaN,Imperial


## Part A3. Bring in and clean, reformat county AMIs data

In [41]:
#bring in 2018 county AMIs file
columns=["County","Income_Category","1","2","4","6","8"]
df = pd.read_csv("2017_amis.csv", delimiter=",", usecols=columns)
df

,County,Income_Category,1,2,4,6,8
0,Alameda County\n4-Per,Extremely Low,21950,25050,31300,36350,41350
1,Alameda County\n4-Per,Very Low Income,36550,41750,52150,60500,68850
2,Alameda County\n4-Per,Low Income,56300,64350,80400,93300,106150
3,Alameda County\n4-Per,Median Income,68200,77900,97400,113000,128550
4,Alameda County\n4-Per,Moderate Income,81850,93500,116900,135600,154300
...,...,...,...,...,...,...,...
285,Yuba County\n4-Person,Extremely Low,12600,16240,24600,32960,39550
286,Yuba County\n4-Person,Very Low Income,21000,24000,29950,34750,39550
287,Yuba County\n4-Person,Low Income,33550,38350,47900,55600,63250
288,Yuba County\n4-Person,Median Income,41950,47900,59900,69500,79050


In [42]:
#clean county names variable
df['County'] = df['County'].str.replace(r' County\n4-Person', '')
df['County'] = df['County'].str.replace(r' County\n4-Perso', '')
df['County'] = df['County'].str.replace(r' County\n4-Pers', '')
df['County'] = df['County'].str.replace(r' County\n4-Per', '')
df['County'] = df['County'].str.replace(r' County\n4-Pe', '')
df['County'] = df['County'].str.replace(r' County\n4-P', '')
df['County'] = df['County'].str.replace(r' County\n4-', '')
df['County'] = df['County'].str.replace(r' County\n4', '')
df['County'] = df['County'].str.replace(r' County\n', '')
df['County'] = df['County'].str.replace(r' County\ ' , '')
df['County'] = df['County'].str.replace(r' County', '')
df['County'] = df['County'].str.replace(r' Count', '')
df['County'] = df['County'].str.replace(r' Coun', '')
df

,County,Income_Category,1,2,4,6,8
0,Alameda,Extremely Low,21950,25050,31300,36350,41350
1,Alameda,Very Low Income,36550,41750,52150,60500,68850
2,Alameda,Low Income,56300,64350,80400,93300,106150
3,Alameda,Median Income,68200,77900,97400,113000,128550
4,Alameda,Moderate Income,81850,93500,116900,135600,154300
...,...,...,...,...,...,...,...
285,Yuba,Extremely Low,12600,16240,24600,32960,39550
286,Yuba,Very Low Income,21000,24000,29950,34750,39550
287,Yuba,Low Income,33550,38350,47900,55600,63250
288,Yuba,Median Income,41950,47900,59900,69500,79050


In [43]:
#create df for each income level and calculate max rents, rename variables
eli_df = df[df.Income_Category =="Extremely Low"].copy()
eli_df["1"]=eli_df["1"]*0.025
eli_df["2"]=eli_df["2"]*0.025
eli_df["4"]=eli_df["4"]*0.025
eli_df["6"]=eli_df["6"]*0.025
eli_df["8"]=eli_df["8"]*0.025
eli_df.rename(columns={"1":"ELI_studio","2":"ELI_1_br","4":"ELI_2_br","6":"ELI_3_br","8":"ELI_4_br"}, inplace=True)
eli_df.drop('Income_Category', axis=1, inplace=True)
eli_df

,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br
0,Alameda,548.75,626.25,782.50,908.75,1033.75
5,Alpine,453.75,518.75,647.50,824.00,1033.00
10,Amador,380.00,435.00,615.00,824.00,1033.00
15,Butte,330.00,406.00,615.00,824.00,1033.00
20,Calaveras,368.75,421.25,615.00,824.00,1033.00
25,Colusa,315.00,406.00,615.00,824.00,988.75
30,Contra Costa,548.75,626.25,782.50,908.75,1033.75
35,Del Norte,315.00,406.00,615.00,824.00,988.75
40,El Dorado,400.00,457.50,615.00,824.00,1033.00
45,Fresno,315.00,406.00,615.00,824.00,988.75


In [44]:
vli_df = df[df.Income_Category =="Very Low Income"].copy()
vli_df["1"]=vli_df["1"]*0.025
vli_df["2"]=vli_df["2"]*0.025
vli_df["4"]=vli_df["4"]*0.025
vli_df["6"]=vli_df["6"]*0.025
vli_df["8"]=vli_df["8"]*0.025
vli_df.rename(columns={"1":"VLI_studio","2":"VLI_1_br","4":"VLI_2_br","6":"VLI_3_br","8":"VLI_4_br"}, inplace=True)
vli_df.drop('Income_Category', axis=1, inplace=True)
vli_df

,County,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br
1,Alameda,913.75,1043.75,1303.75,1512.50,1721.25
6,Alpine,756.25,865.00,1080.00,1253.75,1426.25
11,Amador,633.75,723.75,903.75,1048.75,1193.75
16,Butte,548.75,626.25,782.50,908.75,1033.75
21,Calaveras,615.00,702.50,877.50,1018.75,1158.75
26,Colusa,525.00,600.00,748.75,868.75,988.75
31,Contra Costa,913.75,1043.75,1303.75,1512.50,1721.25
36,Del Norte,525.00,600.00,748.75,868.75,988.75
41,El Dorado,666.25,761.25,951.25,1103.75,1256.25
46,Fresno,525.00,600.00,748.75,868.75,988.75


In [45]:
li_df = df[df.Income_Category =="Low Income"].copy()
li_df["1"]=li_df["1"]*0.025
li_df["2"]=li_df["2"]*0.025
li_df["4"]=li_df["4"]*0.025
li_df["6"]=li_df["6"]*0.025
li_df["8"]=li_df["8"]*0.025
li_df.rename(columns={"1":"LI_studio","2":"LI_1_br","4":"LI_2_br","6":"LI_3_br","8":"LI_4_br"}, inplace=True)
li_df.drop('Income_Category', axis=1, inplace=True)
li_df

,County,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br
2,Alameda,1407.50,1608.75,2010.00,2332.50,2653.75
7,Alpine,1152.50,1316.25,1645.00,1908.75,2172.50
12,Amador,1012.50,1157.50,1446.25,1678.75,1910.00
17,Butte,877.50,1002.50,1252.50,1453.75,1653.75
22,Calaveras,983.75,1123.75,1403.75,1628.75,1853.75
27,Colusa,838.75,958.75,1197.50,1390.00,1581.25
32,Contra Costa,1407.50,1608.75,2010.00,2332.50,2653.75
37,Del Norte,838.75,958.75,1197.50,1390.00,1581.25
42,El Dorado,1066.25,1218.75,1522.50,1766.25,2010.00
47,Fresno,838.75,958.75,1197.50,1390.00,1581.25


In [46]:
mi_df = df[df.Income_Category =="Median Income"].copy()
mi_df["1"]=mi_df["1"]*0.025
mi_df["2"]=mi_df["2"]*0.025
mi_df["4"]=mi_df["4"]*0.025
mi_df["6"]=mi_df["6"]*0.025
mi_df["8"]=mi_df["8"]*0.025
mi_df.rename(columns={"1":"MI_studio","2":"MI_1_br","4":"MI_2_br","6":"MI_3_br","8":"MI_4_br"}, inplace=True)
mi_df.drop('Income_Category', axis=1, inplace=True)
mi_df

,County,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br
3,Alameda,1705.00,1947.50,2435.00,2825.00,3213.75
8,Alpine,1661.25,1897.50,2372.50,2752.50,3131.25
13,Amador,1265.00,1446.25,1807.50,2096.25,2386.25
18,Butte,1095.00,1252.50,1565.00,1815.00,2066.25
23,Calaveras,1228.75,1403.75,1755.00,2036.25,2316.25
28,Colusa,1048.75,1197.50,1497.50,1737.50,1976.25
33,Contra Costa,1705.00,1947.50,2435.00,2825.00,3213.75
38,Del Norte,1048.75,1197.50,1497.50,1737.50,1976.25
43,El Dorado,1331.25,1522.50,1902.50,2207.50,2511.25
48,Fresno,1048.75,1197.50,1497.50,1737.50,1976.25


In [47]:
moi_df = df[df.Income_Category =="Moderate Income"].copy()
moi_df["1"]=moi_df["1"]*0.025
moi_df["2"]=moi_df["2"]*0.025
moi_df["4"]=moi_df["4"]*0.025
moi_df["6"]=moi_df["6"]*0.025
moi_df["8"]=moi_df["8"]*0.025
moi_df.rename(columns={"1":"MoI_studio","2":"MoI_1_br","4":"MoI_2_br","6":"MoI_3_br","8":"MoI_4_br"}, inplace=True)
moi_df.drop('Income_Category', axis=1, inplace=True)
moi_df

,County,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br
4,Alameda,2046.25,2337.50,2922.50,3390.00,3857.50
9,Alpine,1993.75,2277.50,2847.50,3302.50,3758.75
14,Amador,1517.50,1735.00,2168.75,2516.25,2862.50
19,Butte,1313.75,1502.50,1877.50,2177.50,2478.75
24,Calaveras,1473.75,1685.00,2106.25,2443.75,2780.00
29,Colusa,1258.75,1437.50,1797.50,2085.00,2372.50
34,Contra Costa,2046.25,2337.50,2922.50,3390.00,3857.50
39,Del Norte,1258.75,1437.50,1797.50,2085.00,2372.50
44,El Dorado,1597.50,1826.25,2282.50,2647.50,3012.50
49,Fresno,1258.75,1437.50,1797.50,2085.00,2372.50


In [48]:
#combine dfs
all_amis_df=eli_df.merge(vli_df, how='left', left_on = "County",right_on = "County")
all_amis_df=all_amis_df.merge(li_df, how='left', left_on = "County",right_on = "County")
all_amis_df=all_amis_df.merge(mi_df, how='left', left_on = "County",right_on = "County")
all_amis_df=all_amis_df.merge(moi_df, how='left', left_on = "County",right_on = "County")
all_amis_df

,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br
0,Alameda,548.75,626.25,782.50,908.75,1033.75,913.75,1043.75,1303.75,1512.50,1721.25,1407.50,1608.75,2010.00,2332.50,2653.75,1705.00,1947.50,2435.00,2825.00,3213.75,2046.25,2337.50,2922.50,3390.00,3857.50
1,Alpine,453.75,518.75,647.50,824.00,1033.00,756.25,865.00,1080.00,1253.75,1426.25,1152.50,1316.25,1645.00,1908.75,2172.50,1661.25,1897.50,2372.50,2752.50,3131.25,1993.75,2277.50,2847.50,3302.50,3758.75
2,Amador,380.00,435.00,615.00,824.00,1033.00,633.75,723.75,903.75,1048.75,1193.75,1012.50,1157.50,1446.25,1678.75,1910.00,1265.00,1446.25,1807.50,2096.25,2386.25,1517.50,1735.00,2168.75,2516.25,2862.50
3,Butte,330.00,406.00,615.00,824.00,1033.00,548.75,626.25,782.50,908.75,1033.75,877.50,1002.50,1252.50,1453.75,1653.75,1095.00,1252.50,1565.00,1815.00,2066.25,1313.75,1502.50,1877.50,2177.50,2478.75
4,Calaveras,368.75,421.25,615.00,824.00,1033.00,615.00,702.50,877.50,1018.75,1158.75,983.75,1123.75,1403.75,1628.75,1853.75,1228.75,1403.75,1755.00,2036.25,2316.25,1473.75,1685.00,2106.25,2443.75,2780.00
5,Colusa,315.00,406.00,615.00,824.00,988.75,525.00,600.00,748.75,868.75,988.75,838.75,958.75,1197.50,1390.00,1581.25,1048.75,1197.50,1497.50,1737.50,1976.25,1258.75,1437.50,1797.50,2085.00,2372.50
6,Contra Costa,548.75,626.25,782.50,908.75,1033.75,913.75,1043.75,1303.75,1512.50,1721.25,1407.50,1608.75,2010.00,2332.50,2653.75,1705.00,1947.50,2435.00,2825.00,3213.75,2046.25,2337.50,2922.50,3390.00,3857.50
7,Del Norte,315.00,406.00,615.00,824.00,988.75,525.00,600.00,748.75,868.75,988.75,838.75,958.75,1197.50,1390.00,1581.25,1048.75,1197.50,1497.50,1737.50,1976.25,1258.75,1437.50,1797.50,2085.00,2372.50
8,El Dorado,400.00,457.50,615.00,824.00,1033.00,666.25,761.25,951.25,1103.75,1256.25,1066.25,1218.75,1522.50,1766.25,2010.00,1331.25,1522.50,1902.50,2207.50,2511.25,1597.50,1826.25,2282.50,2647.50,3012.50
9,Fresno,315.00,406.00,615.00,824.00,988.75,525.00,600.00,748.75,868.75,988.75,838.75,958.75,1197.50,1390.00,1581.25,1048.75,1197.50,1497.50,1737.50,1976.25,1258.75,1437.50,1797.50,2085.00,2372.50


## Part A4. Add count dummies for each unit based on income and unit type

In [49]:
# merge datasets
units_df=puma_county_df.merge(all_amis_df, how='left', left_on = "County",right_on = "County")
units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br
0,H,6703,59.00,2.00,9.00,nan,nan,940.00,nan,3.00,nan,nan,990.00,25.00,47000.00,3.00,nan,Sacramento CA,NaN,NaN,NaN,NaN,NaN,NaN,Sacramento,400.00,457.50,615.00,824.00,1033.00,666.25,761.25,951.25,1103.75,1256.25,1066.25,1218.75,1522.50,1766.25,2010.00,1331.25,1522.50,1902.50,2207.50,2511.25,1597.50,1826.25,2282.50,2647.50,3012.50
1,H,110,74.00,4.00,2.00,0.00,2600.00,nan,nan,1.00,nan,780000.00,nan,nan,130500.00,1.00,67.00,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN,Alameda,548.75,626.25,782.50,908.75,1033.75,913.75,1043.75,1303.75,1512.50,1721.25,1407.50,1608.75,2010.00,2332.50,2653.75,1705.00,1947.50,2435.00,2825.00,3213.75,2046.25,2337.50,2922.50,3390.00,3857.50
2,H,7901,111.00,2.00,6.00,nan,nan,nan,nan,nan,5.00,nan,nan,nan,nan,nan,nan,San Luis Obispo CA,NaN,NaN,NaN,NaN,NaN,NaN,San Luis Obispo,428.75,490.00,615.00,824.00,1033.00,715.00,817.50,1021.25,1185.00,1348.75,1143.75,1307.50,1633.75,1896.25,2157.50,1456.25,1663.75,2080.00,2412.50,2745.00,1747.50,1997.50,2496.25,2896.25,3295.00
3,H,1906,63.00,3.00,2.00,0.00,nan,nan,nan,2.00,nan,175000.00,nan,nan,13530.00,7.00,14.00,Fresno CA,NaN,NaN,NaN,NaN,NaN,NaN,Fresno,315.00,406.00,615.00,824.00,988.75,525.00,600.00,748.75,868.75,988.75,838.75,958.75,1197.50,1390.00,1581.25,1048.75,1197.50,1497.50,1737.50,1976.25,1258.75,1437.50,1797.50,2085.00,2372.50
4,H,6710,75.00,5.00,2.00,0.00,nan,nan,nan,2.00,nan,500000.00,nan,nan,104000.00,4.00,48.00,Sacramento CA,NaN,NaN,NaN,NaN,NaN,NaN,Sacramento,400.00,457.50,615.00,824.00,1033.00,666.25,761.25,951.25,1103.75,1256.25,1066.25,1218.75,1522.50,1766.25,2010.00,1331.25,1522.50,1902.50,2207.50,2511.25,1597.50,1826.25,2282.50,2647.50,3012.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157729,H,6710,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Sacramento CA,NaN,NaN,NaN,NaN,NaN,NaN,Sacramento,400.00,457.50,615.00,824.00,1033.00,666.25,761.25,951.25,1103.75,1256.25,1066.25,1218.75,1522.50,1766.25,2010.00,1331.25,1522.50,1902.50,2207.50,2511.25,1597.50,1826.25,2282.50,2647.50,3012.50
157730,H,2901,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Kern CA,NaN,NaN,NaN,NaN,NaN,NaN,Kern,315.00,406.00,615.00,824.00,988.75,525.00,600.00,748.75,868.75,988.75,838.75,958.75,1197.50,1390.00,1581.25,1048.75,1197.50,1497.50,1737.50,1976.25,1258.75,1437.50,1797.50,2085.00,2372.50
157731,H,3900,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Madera CA,NaN,NaN,NaN,NaN,NaN,NaN,Madera,315.00,406.00,615.00,824.00,988.75,525.00,600.00,748.75,868.75,988.75,838.75,958.75,1197.50,1390.00,1581.25,1048.75,1197.50,1497.50,1737.50,1976.25,1258.75,1437.50,1797.50,2085.00,2372.50
157732,H,2500,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Imperial CA,NaN,NaN,NaN,NaN,NaN,NaN,Imperial,315.00,406.00,615.00,824.00,988.75,525.00,600.00,748.75,868.75,988.75,838.75,958.75,1197.50,1390.00,1581.25,1048.75,1197.50,1497.50,1737.50,1976.25,1258.75,1437.50,1797.50,2085.00,2372.50


In [50]:
# create filter masks for each affordability and unit type
ELI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['monthly_rent']<=units_df["ELI_studio"])
ELI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['monthly_rent']<=units_df["ELI_1_br"])
ELI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['monthly_rent']<=units_df["ELI_2_br"])
ELI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['monthly_rent']<=units_df["ELI_3_br"])
ELI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['monthly_rent']<=units_df["ELI_4_br"])
VLI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['monthly_rent']<=units_df["VLI_studio"])
VLI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['monthly_rent']<=units_df["VLI_1_br"])
VLI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['monthly_rent']<=units_df["VLI_2_br"])
VLI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['monthly_rent']<=units_df["VLI_3_br"])
VLI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['monthly_rent']<=units_df["VLI_4_br"])
LI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['monthly_rent']<=units_df["LI_studio"])
LI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['monthly_rent']<=units_df["LI_1_br"])
LI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['monthly_rent']<=units_df["LI_2_br"])
LI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['monthly_rent']<=units_df["LI_3_br"])
LI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['monthly_rent']<=units_df["LI_4_br"])
MI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['monthly_rent']<=units_df["MI_studio"])
MI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['monthly_rent']<=units_df["MI_1_br"])
MI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['monthly_rent']<=units_df["MI_2_br"])
MI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['monthly_rent']<=units_df["MI_3_br"])
MI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['monthly_rent']<=units_df["MI_4_br"])
MoI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['monthly_rent']<=units_df["MoI_studio"])
MoI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['monthly_rent']<=units_df["MoI_1_br"])
MoI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['monthly_rent']<=units_df["MoI_2_br"])
MoI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['monthly_rent']<=units_df["MoI_3_br"])
MoI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['monthly_rent']<=units_df["MoI_4_br"])
HI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['monthly_rent']>=units_df["MoI_studio"])
HI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['monthly_rent']>=units_df["MoI_1_br"])
HI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['monthly_rent']>=units_df["MoI_2_br"])
HI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['monthly_rent']>=units_df["MoI_3_br"])
HI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['monthly_rent']>=units_df["MoI_4_br"])

# apply filter masks to create dummy variables
units_df['rent_ELI_studio_count']=np.where(ELI_studio_mask,1,0)
units_df['rent_ELI_1_br_count']=np.where(ELI_1_br_mask,1,0)
units_df['rent_ELI_2_br_count']=np.where(ELI_2_br_mask,1,0)
units_df['rent_ELI_3_br_count']=np.where(ELI_3_br_mask,1,0)
units_df['rent_ELI_4_br_count']=np.where(ELI_4_br_mask,1,0)
units_df['rent_VLI_studio_count']=np.where(VLI_studio_mask,1,0)
units_df['rent_VLI_1_br_count']=np.where(VLI_1_br_mask,1,0)
units_df['rent_VLI_2_br_count']=np.where(VLI_2_br_mask,1,0)
units_df['rent_VLI_3_br_count']=np.where(VLI_3_br_mask,1,0)
units_df['rent_VLI_4_br_count']=np.where(VLI_4_br_mask,1,0)
units_df['rent_LI_studio_count']=np.where(LI_studio_mask,1,0)
units_df['rent_LI_1_br_count']=np.where(LI_1_br_mask,1,0)
units_df['rent_LI_2_br_count']=np.where(LI_2_br_mask,1,0)
units_df['rent_LI_3_br_count']=np.where(LI_3_br_mask,1,0)
units_df['rent_LI_4_br_count']=np.where(LI_4_br_mask,1,0)
units_df['rent_MI_studio_count']=np.where(MI_studio_mask,1,0)
units_df['rent_MI_1_br_count']=np.where(MI_1_br_mask,1,0)
units_df['rent_MI_2_br_count']=np.where(MI_2_br_mask,1,0)
units_df['rent_MI_3_br_count']=np.where(MI_3_br_mask,1,0)
units_df['rent_MI_4_br_count']=np.where(MI_4_br_mask,1,0)
units_df['rent_MoI_studio_count']=np.where(MoI_studio_mask,1,0)
units_df['rent_MoI_1_br_count']=np.where(MoI_1_br_mask,1,0)
units_df['rent_MoI_2_br_count']=np.where(MoI_2_br_mask,1,0)
units_df['rent_MoI_3_br_count']=np.where(MoI_3_br_mask,1,0)
units_df['rent_MoI_4_br_count']=np.where(MoI_4_br_mask,1,0)
units_df['rent_HI_studio_count']=np.where(HI_studio_mask,1,0)
units_df['rent_HI_1_br_count']=np.where(HI_1_br_mask,1,0)
units_df['rent_HI_2_br_count']=np.where(HI_2_br_mask,1,0)
units_df['rent_HI_3_br_count']=np.where(HI_3_br_mask,1,0)
units_df['rent_HI_4_br_count']=np.where(HI_4_br_mask,1,0)
units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,rent_ELI_studio_count,rent_ELI_1_br_count,rent_ELI_2_br_count,rent_ELI_3_br_count,rent_ELI_4_br_count,rent_VLI_studio_count,rent_VLI_1_br_count,rent_VLI_2_br_count,rent_VLI_3_br_count,rent_VLI_4_br_count,rent_LI_studio_count,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count
0,H,6703,59.00,2.00,9.00,nan,nan,940.00,nan,3.00,nan,nan,990.00,25.00,47000.00,3.00,nan,Sacramento CA,NaN,NaN,NaN,NaN,NaN,NaN,Sacramento,400.00,457.50,615.00,824.00,1033.00,666.25,761.25,951.25,1103.75,1256.25,1066.25,1218.75,1522.50,1766.25,2010.00,1331.25,1522.50,1902.50,2207.50,2511.25,1597.50,1826.25,2282.50,2647.50,3012.50,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0
1,H,110,74.00,4.00,2.00,0.00,2600.00,nan,nan,1.00,nan,780000.00,nan,nan,130500.00,1.00,67.00,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN,Alameda,548.75,626.25,782.50,908.75,1033.75,913.75,1043.75,1303.75,1512.50,1721.25,1407.50,1608.75,2010.00,2332.50,2653.75,1705.00,1947.50,2435.00,2825.00,3213.75,2046.25,2337.50,2922.50,3390.00,3857.50,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,H,7901,111.00,2.00,6.00,nan,nan,nan,nan,nan,5.00,nan,nan,nan,nan,nan,nan,San Luis Obispo CA,NaN,NaN,NaN,NaN,NaN,NaN,San Luis Obispo,428.75,490.00,615.00,824.00,1033.00,715.00,817.50,1021.25,1185.00,1348.75,1143.75,1307.50,1633.75,1896.25,2157.50,1456.25,1663.75,2080.00,2412.50,2745.00,1747.50,1997.50,2496.25,2896.25,3295.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,H,1906,63.00,3.00,2.00,0.00,nan,nan,nan,2.00,nan,175000.00,nan,nan,13530.00,7.00,14.00,Fresno CA,NaN,NaN,NaN,NaN,NaN,NaN,Fresno,315.00,406.00,615.00,824.00,988.75,525.00,600.00,748.75,868.75,988.75,838.75,958.75,1197.50,1390.00,1581.25,1048.75,1197.50,1497.50,1737.50,1976.25,1258.75,1437.50,1797.50,2085.00,2372.50,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,H,6710,75.00,5.00,2.00,0.00,nan,nan,nan,2.00,nan,500000.00,nan,nan,104000.00,4.00,48.00,Sacramento CA,NaN,NaN,NaN,NaN,NaN,NaN,Sacramento,400.00,457.50,615.00,824.00,1033.00,666.25,761.25,951.25,1103.75,1256.25,1066.25,1218.75,1522.50,1766.25,2010.00,1331.25,1522.50,1902.50,2207.50,2511.25,1597.50,1826.25,2282.50,2647.50,3012.50,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157729,H,6710,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Sacramento CA,NaN,NaN,NaN,NaN,NaN,NaN,Sacramento,400.00,457.50,615.00,824.00,1033.00,666.25,761.25,951.25,1103.75,1256.25,1066.25,1218.75,1522.50,1766.25,2010.00,1331.25,1522.50,1902.50,2207.50,2511.25,1597.50,1826.25,2282.50,2647.50,3012.50,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
157730,H,2901,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Kern CA,NaN,NaN,NaN,NaN,NaN,NaN,Kern,315.00,406.00,615.00,824.00,988.75,525.00,600.00,748.75,868.75,988.75,838.75,958.75,1197.50,1390.00,1581.25,1048.75,1197.50,1497.50,1737.50,1976.25,

## Part B1. Calculate total monthly housing cost by adding monthly first mortgage, second/junior mortgage, and condo fees

In [51]:
units_df['own_monthly_cost']=units_df['first_mortgage'] + units_df['second_mortgage'] + units_df['condo_fee'] + (units_df['prop_tax']/12)
units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,rent_ELI_studio_count,rent_ELI_1_br_count,rent_ELI_2_br_count,rent_ELI_3_br_count,rent_ELI_4_br_count,rent_VLI_studio_count,rent_VLI_1_br_count,rent_VLI_2_br_count,rent_VLI_3_br_count,rent_VLI_4_br_count,rent_LI_studio_count,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_monthly_cost
0,H,6703,59.00,2.00,9.00,nan,nan,940.00,nan,3.00,nan,nan,990.00,25.00,47000.00,3.00,nan,Sacramento CA,NaN,NaN,NaN,NaN,NaN,NaN,Sacramento,400.00,457.50,615.00,824.00,1033.00,666.25,761.25,951.25,1103.75,1256.25,1066.25,1218.75,1522.50,1766.25,2010.00,1331.25,1522.50,1902.50,2207.50,2511.25,1597.50,1826.25,2282.50,2647.50,3012.50,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,nan
1,H,110,74.00,4.00,2.00,0.00,2600.00,nan,nan,1.00,nan,780000.00,nan,nan,130500.00,1.00,67.00,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN,Alameda,548.75,626.25,782.50,908.75,1033.75,913.75,1043.75,1303.75,1512.50,1721.25,1407.50,1608.75,2010.00,2332.50,2653.75,1705.00,1947.50,2435.00,2825.00,3213.75,2046.25,2337.50,2922.50,3390.00,3857.50,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan
2,H,7901,111.00,2.00,6.00,nan,nan,nan,nan,nan,5.00,nan,nan,nan,nan,nan,nan,San Luis Obispo CA,NaN,NaN,NaN,NaN,NaN,NaN,San Luis Obispo,428.75,490.00,615.00,824.00,1033.00,715.00,817.50,1021.25,1185.00,1348.75,1143.75,1307.50,1633.75,1896.25,2157.50,1456.25,1663.75,2080.00,2412.50,2745.00,1747.50,1997.50,2496.25,2896.25,3295.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan
3,H,1906,63.00,3.00,2.00,0.00,nan,nan,nan,2.00,nan,175000.00,nan,nan,13530.00,7.00,14.00,Fresno CA,NaN,NaN,NaN,NaN,NaN,NaN,Fresno,315.00,406.00,615.00,824.00,988.75,525.00,600.00,748.75,868.75,988.75,838.75,958.75,1197.50,1390.00,1581.25,1048.75,1197.50,1497.50,1737.50,1976.25,1258.75,1437.50,1797.50,2085.00,2372.50,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan
4,H,6710,75.00,5.00,2.00,0.00,nan,nan,nan,2.00,nan,500000.00,nan,nan,104000.00,4.00,48.00,Sacramento CA,NaN,NaN,NaN,NaN,NaN,NaN,Sacramento,400.00,457.50,615.00,824.00,1033.00,666.25,761.25,951.25,1103.75,1256.25,1066.25,1218.75,1522.50,1766.25,2010.00,1331.25,1522.50,1902.50,2207.50,2511.25,1597.50,1826.25,2282.50,2647.50,3012.50,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157729,H,6710,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Sacramento CA,NaN,NaN,NaN,NaN,NaN,NaN,Sacramento,400.00,457.50,615.00,824.00,1033.00,666.25,761.25,951.25,1103.75,1256.25,1066.25,1218.75,1522.50,1766.25,2010.00,1331.25,1522.50,1902.50,2207.50,2511.25,1597.50,1826.25,2282.50,2647.50,3012.50,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan
157730,H,2901,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Kern CA,NaN,NaN,NaN,NaN,NaN,NaN,Kern,315.00,406.00,615.00,824.00,988.75,525.00,600.00,748.75,868.75,988.75,838.75,958.75,1197.50,1390.00,158

# Part B2. Add count dummies for each unit based on income and unit type

In [52]:
# create filter masks for each affordability and unit type
ELI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['own_monthly_cost']<=units_df["ELI_studio"])
ELI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['own_monthly_cost']<=units_df["ELI_1_br"])
ELI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['own_monthly_cost']<=units_df["ELI_2_br"])
ELI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['own_monthly_cost']<=units_df["ELI_3_br"])
ELI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['own_monthly_cost']<=units_df["ELI_4_br"])
VLI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['own_monthly_cost']<=units_df["VLI_studio"])
VLI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['own_monthly_cost']<=units_df["VLI_1_br"])
VLI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['own_monthly_cost']<=units_df["VLI_2_br"])
VLI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['own_monthly_cost']<=units_df["VLI_3_br"])
VLI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['own_monthly_cost']<=units_df["VLI_4_br"])
LI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['own_monthly_cost']<=units_df["LI_studio"])
LI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['own_monthly_cost']<=units_df["LI_1_br"])
LI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['own_monthly_cost']<=units_df["LI_2_br"])
LI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['own_monthly_cost']<=units_df["LI_3_br"])
LI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['own_monthly_cost']<=units_df["LI_4_br"])
MI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['own_monthly_cost']<=units_df["MI_studio"])
MI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['own_monthly_cost']<=units_df["MI_1_br"])
MI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['own_monthly_cost']<=units_df["MI_2_br"])
MI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['own_monthly_cost']<=units_df["MI_3_br"])
MI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['own_monthly_cost']<=units_df["MI_4_br"])
MoI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['own_monthly_cost']<=units_df["MoI_studio"])
MoI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['own_monthly_cost']<=units_df["MoI_1_br"])
MoI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['own_monthly_cost']<=units_df["MoI_2_br"])
MoI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['own_monthly_cost']<=units_df["MoI_3_br"])
MoI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['own_monthly_cost']<=units_df["MoI_4_br"])
HI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['own_monthly_cost']>=units_df["MoI_studio"])
HI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['own_monthly_cost']>=units_df["MoI_1_br"])
HI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['own_monthly_cost']>=units_df["MoI_2_br"])
HI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['own_monthly_cost']>=units_df["MoI_3_br"])
HI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['own_monthly_cost']>=units_df["MoI_4_br"])

# apply filter masks to create dummy variables
units_df['own_ELI_studio_count']=np.where(ELI_studio_mask,1,0)
units_df['own_ELI_1_br_count']=np.where(ELI_1_br_mask,1,0)
units_df['own_ELI_2_br_count']=np.where(ELI_2_br_mask,1,0)
units_df['own_ELI_3_br_count']=np.where(ELI_3_br_mask,1,0)
units_df['own_ELI_4_br_count']=np.where(ELI_4_br_mask,1,0)
units_df['own_VLI_studio_count']=np.where(VLI_studio_mask,1,0)
units_df['own_VLI_1_br_count']=np.where(VLI_1_br_mask,1,0)
units_df['own_VLI_2_br_count']=np.where(VLI_2_br_mask,1,0)
units_df['own_VLI_3_br_count']=np.where(VLI_3_br_mask,1,0)
units_df['own_VLI_4_br_count']=np.where(VLI_4_br_mask,1,0)
units_df['own_LI_studio_count']=np.where(LI_studio_mask,1,0)
units_df['own_LI_1_br_count']=np.where(LI_1_br_mask,1,0)
units_df['own_LI_2_br_count']=np.where(LI_2_br_mask,1,0)
units_df['own_LI_3_br_count']=np.where(LI_3_br_mask,1,0)
units_df['own_LI_4_br_count']=np.where(LI_4_br_mask,1,0)
units_df['own_MI_studio_count']=np.where(MI_studio_mask,1,0)
units_df['own_MI_1_br_count']=np.where(MI_1_br_mask,1,0)
units_df['own_MI_2_br_count']=np.where(MI_2_br_mask,1,0)
units_df['own_MI_3_br_count']=np.where(MI_3_br_mask,1,0)
units_df['own_MI_4_br_count']=np.where(MI_4_br_mask,1,0)
units_df['own_MoI_studio_count']=np.where(MoI_studio_mask,1,0)
units_df['own_MoI_1_br_count']=np.where(MoI_1_br_mask,1,0)
units_df['own_MoI_2_br_count']=np.where(MoI_2_br_mask,1,0)
units_df['own_MoI_3_br_count']=np.where(MoI_3_br_mask,1,0)
units_df['own_MoI_4_br_count']=np.where(MoI_4_br_mask,1,0)
units_df['own_HI_studio_count']=np.where(HI_studio_mask,1,0)
units_df['own_HI_1_br_count']=np.where(HI_1_br_mask,1,0)
units_df['own_HI_2_br_count']=np.where(HI_2_br_mask,1,0)
units_df['own_HI_3_br_count']=np.where(HI_3_br_mask,1,0)
units_df['own_HI_4_br_count']=np.where(HI_4_br_mask,1,0)
units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,...,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_monthly_cost,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count
0,H,6703,59.00,2.00,9.00,nan,nan,940.00,nan,3.00,nan,nan,990.00,25.00,47000.00,3.00,nan,Sacramento CA,NaN,NaN,NaN,NaN,NaN,NaN,Sacramento,400.00,457.50,615.00,824.00,1033.00,666.25,761.25,951.25,1103.75,1256.25,1066.25,1218.75,1522.50,1766.25,2010.00,1331.25,1522.50,1902.50,2207.50,2511.25,1597.50,1826.25,2282.50,2647.50,3012.50,...,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,H,110,74.00,4.00,2.00,0.00,2600.00,nan,nan,1.00,nan,780000.00,nan,nan,130500.00,1.00,67.00,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN,Alameda,548.75,626.25,782.50,908.75,1033.75,913.75,1043.75,1303.75,1512.50,1721.25,1407.50,1608.75,2010.00,2332.50,2653.75,1705.00,1947.50,2435.00,2825.00,3213.75,2046.25,2337.50,2922.50,3390.00,3857.50,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,H,7901,111.00,2.00,6.00,nan,nan,nan,nan,nan,5.00,nan,nan,nan,nan,nan,nan,San Luis Obispo CA,NaN,NaN,NaN,NaN,NaN,NaN,San Luis Obispo,428.75,490.00,615.00,824.00,1033.00,715.00,817.50,1021.25,1185.00,1348.75,1143.75,1307.50,1633.75,1896.25,2157.50,1456.25,1663.75,2080.00,2412.50,2745.00,1747.50,1997.50,2496.25,2896.25,3295.00,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,H,1906,63.00,3.00,2.00,0.00,nan,nan,nan,2.00,nan,175000.00,nan,nan,13530.00,7.00,14.00,Fresno CA,NaN,NaN,NaN,NaN,NaN,NaN,Fresno,315.00,406.00,615.00,824.00,988.75,525.00,600.00,748.75,868.75,988.75,838.75,958.75,1197.50,1390.00,1581.25,1048.75,1197.50,1497.50,1737.50,1976.25,1258.75,1437.50,1797.50,2085.00,2372.50,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,H,6710,75.00,5.00,2.00,0.00,nan,nan,nan,2.00,nan,500000.00,nan,nan,104000.00,4.00,48.00,Sacramento CA,NaN,NaN,NaN,NaN,NaN,NaN,Sacramento,400.00,457.50,615.00,824.00,1033.00,666.25,761.25,951.25,1103.75,1256.25,1066.25,1218.75,1522.50,1766.25,2010.00,1331.25,1522.50,1902.50,2207.50,2511.25,1597.50,1826.25,2282.50,2647.50,3012.50,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

# Part C1. Multiply each dummy by the the household weight variable

In [53]:
units_df.rent_ELI_studio_count=units_df.rent_ELI_studio_count*units_df.weight
units_df.rent_ELI_1_br_count=units_df.rent_ELI_1_br_count*units_df.weight
units_df.rent_ELI_2_br_count=units_df.rent_ELI_2_br_count*units_df.weight
units_df.rent_ELI_3_br_count=units_df.rent_ELI_3_br_count*units_df.weight
units_df.rent_ELI_4_br_count=units_df.rent_ELI_4_br_count*units_df.weight
units_df.rent_VLI_studio_count=units_df.rent_VLI_studio_count*units_df.weight
units_df.rent_VLI_1_br_count=units_df.rent_VLI_1_br_count*units_df.weight
units_df.rent_VLI_2_br_count=units_df.rent_VLI_2_br_count*units_df.weight
units_df.rent_VLI_3_br_count=units_df.rent_VLI_3_br_count*units_df.weight
units_df.rent_VLI_4_br_count=units_df.rent_VLI_4_br_count*units_df.weight
units_df.rent_LI_studio_count=units_df.rent_LI_studio_count*units_df.weight
units_df.rent_LI_1_br_count=units_df.rent_LI_1_br_count*units_df.weight
units_df.rent_LI_2_br_count=units_df.rent_LI_2_br_count*units_df.weight
units_df.rent_LI_3_br_count=units_df.rent_LI_3_br_count*units_df.weight
units_df.rent_LI_4_br_count=units_df.rent_LI_4_br_count*units_df.weight
units_df.rent_MI_studio_count=units_df.rent_MI_studio_count*units_df.weight
units_df.rent_MI_1_br_count=units_df.rent_MI_1_br_count*units_df.weight
units_df.rent_MI_2_br_count=units_df.rent_MI_2_br_count*units_df.weight
units_df.rent_MI_3_br_count=units_df.rent_MI_3_br_count*units_df.weight
units_df.rent_MI_4_br_count=units_df.rent_MI_4_br_count*units_df.weight
units_df.rent_MoI_studio_count=units_df.rent_MoI_studio_count*units_df.weight
units_df.rent_MoI_1_br_count=units_df.rent_MoI_1_br_count*units_df.weight
units_df.rent_MoI_2_br_count=units_df.rent_MoI_2_br_count*units_df.weight
units_df.rent_MoI_3_br_count=units_df.rent_MoI_3_br_count*units_df.weight
units_df.rent_MoI_4_br_count=units_df.rent_MoI_4_br_count*units_df.weight
units_df.rent_HI_studio_count=units_df.rent_HI_studio_count*units_df.weight
units_df.rent_HI_1_br_count=units_df.rent_HI_1_br_count*units_df.weight
units_df.rent_HI_2_br_count=units_df.rent_HI_2_br_count*units_df.weight
units_df.rent_HI_3_br_count=units_df.rent_HI_3_br_count*units_df.weight
units_df.rent_HI_4_br_count=units_df.rent_HI_4_br_count*units_df.weight
units_df.own_ELI_studio_count=units_df.own_ELI_studio_count*units_df.weight
units_df.own_ELI_1_br_count=units_df.own_ELI_1_br_count*units_df.weight
units_df.own_ELI_2_br_count=units_df.own_ELI_2_br_count*units_df.weight
units_df.own_ELI_3_br_count=units_df.own_ELI_3_br_count*units_df.weight
units_df.own_ELI_4_br_count=units_df.own_ELI_4_br_count*units_df.weight
units_df.own_VLI_studio_count=units_df.own_VLI_studio_count*units_df.weight
units_df.own_VLI_1_br_count=units_df.own_VLI_1_br_count*units_df.weight
units_df.own_VLI_2_br_count=units_df.own_VLI_2_br_count*units_df.weight
units_df.own_VLI_3_br_count=units_df.own_VLI_3_br_count*units_df.weight
units_df.own_VLI_4_br_count=units_df.own_VLI_4_br_count*units_df.weight
units_df.own_LI_studio_count=units_df.own_LI_studio_count*units_df.weight
units_df.own_LI_1_br_count=units_df.own_LI_1_br_count*units_df.weight
units_df.own_LI_2_br_count=units_df.own_LI_2_br_count*units_df.weight
units_df.own_LI_3_br_count=units_df.own_LI_3_br_count*units_df.weight
units_df.own_LI_4_br_count=units_df.own_LI_4_br_count*units_df.weight
units_df.own_MI_studio_count=units_df.own_MI_studio_count*units_df.weight
units_df.own_MI_1_br_count=units_df.own_MI_1_br_count*units_df.weight
units_df.own_MI_2_br_count=units_df.own_MI_2_br_count*units_df.weight
units_df.own_MI_3_br_count=units_df.own_MI_3_br_count*units_df.weight
units_df.own_MI_4_br_count=units_df.own_MI_4_br_count*units_df.weight
units_df.own_MoI_studio_count=units_df.own_MoI_studio_count*units_df.weight
units_df.own_MoI_1_br_count=units_df.own_MoI_1_br_count*units_df.weight
units_df.own_MoI_2_br_count=units_df.own_MoI_2_br_count*units_df.weight
units_df.own_MoI_3_br_count=units_df.own_MoI_3_br_count*units_df.weight
units_df.own_MoI_4_br_count=units_df.own_MoI_4_br_count*units_df.weight
units_df.own_HI_studio_count=units_df.own_HI_studio_count*units_df.weight
units_df.own_HI_1_br_count=units_df.own_HI_1_br_count*units_df.weight
units_df.own_HI_2_br_count=units_df.own_HI_2_br_count*units_df.weight
units_df.own_HI_3_br_count=units_df.own_HI_3_br_count*units_df.weight
units_df.own_HI_4_br_count=units_df.own_HI_4_br_count*units_df.weight
units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,...,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_monthly_cost,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count
0,H,6703,59.00,2.00,9.00,nan,nan,940.00,nan,3.00,nan,nan,990.00,25.00,47000.00,3.00,nan,Sacramento CA,NaN,NaN,NaN,NaN,NaN,NaN,Sacramento,400.00,457.50,615.00,824.00,1033.00,666.25,761.25,951.25,1103.75,1256.25,1066.25,1218.75,1522.50,1766.25,2010.00,1331.25,1522.50,1902.50,2207.50,2511.25,1597.50,1826.25,2282.50,2647.50,3012.50,...,0.00,59.00,0.00,0.00,0.00,0.00,59.00,0.00,0.00,0.00,0.00,59.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,H,110,74.00,4.00,2.00,0.00,2600.00,nan,nan,1.00,nan,780000.00,nan,nan,130500.00,1.00,67.00,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN,Alameda,548.75,626.25,782.50,908.75,1033.75,913.75,1043.75,1303.75,1512.50,1721.25,1407.50,1608.75,2010.00,2332.50,2653.75,1705.00,1947.50,2435.00,2825.00,3213.75,2046.25,2337.50,2922.50,3390.00,3857.50,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,H,7901,111.00,2.00,6.00,nan,nan,nan,nan,nan,5.00,nan,nan,nan,nan,nan,nan,San Luis Obispo CA,NaN,NaN,NaN,NaN,NaN,NaN,San Luis Obispo,428.75,490.00,615.00,824.00,1033.00,715.00,817.50,1021.25,1185.00,1348.75,1143.75,1307.50,1633.75,1896.25,2157.50,1456.25,1663.75,2080.00,2412.50,2745.00,1747.50,1997.50,2496.25,2896.25,3295.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,H,1906,63.00,3.00,2.00,0.00,nan,nan,nan,2.00,nan,175000.00,nan,nan,13530.00,7.00,14.00,Fresno CA,NaN,NaN,NaN,NaN,NaN,NaN,Fresno,315.00,406.00,615.00,824.00,988.75,525.00,600.00,748.75,868.75,988.75,838.75,958.75,1197.50,1390.00,1581.25,1048.75,1197.50,1497.50,1737.50,1976.25,1258.75,1437.50,1797.50,2085.00,2372.50,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,H,6710,75.00,5.00,2.00,0.00,nan,nan,nan,2.00,nan,500000.00,nan,nan,104000.00,4.00,48.00,Sacramento CA,NaN,NaN,NaN,NaN,NaN,NaN,Sacramento,400.00,457.50,615.00,824.00,1033.00,666.25,761.25,951.25,1103.75,1256.25,106

In [54]:
units_df[units_df.rent_HI_4_br_count>0]

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,...,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_monthly_cost,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count
526,H,7308,67.00,4.00,2.00,nan,nan,3200.00,nan,3.00,nan,nan,3470.00,19.00,215700.00,3.00,nan,San Diego CA,NaN,NaN,NaN,NaN,NaN,NaN,San Diego,477.50,545.00,681.25,824.00,1033.00,796.25,910.00,1136.25,1318.75,1500.00,1273.75,1455.00,1818.75,2110.00,2401.25,1387.50,1586.25,1982.50,2300.00,2617.50,1665.00,1902.50,2378.75,2758.75,3140.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,67.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1116,H,8303,235.00,4.00,2.00,nan,nan,3800.00,nan,3.00,nan,nan,4330.00,101.00,18500.00,4.00,nan,Santa Barbara CA,NaN,NaN,NaN,NaN,NaN,NaN,Santa Barbara,472.50,540.00,675.00,824.00,1033.00,787.50,900.00,1125.00,1305.00,1485.00,1261.25,1441.25,1801.25,2090.00,2378.75,1348.75,1542.50,1927.50,2236.25,2543.75,1618.75,1850.00,2312.50,2682.50,3052.50,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,235.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1233,H,3751,140.00,4.00,3.00,nan,nan,2600.00,nan,3.00,nan,nan,2718.00,40.00,81000.00,3.00,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,473.75,541.25,676.25,824.00,1033.00,788.75,901.25,1126.25,1307.50,1487.50,1262.50,1442.50,1802.50,2091.25,2380.00,1133.75,1296.25,1620.00,1878.75,2138.75,1361.25,1555.00,1943.75,2255.00,2566.25,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,140.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3384,H,6502,79.00,4.00,2.00,nan,nan,3800.00,nan,3.00,nan,nan,4360.00,28.00,185000.00,2.00,nan,Riverside CA,NaN,NaN,NaN,NaN,NaN,NaN,Riverside,352.50,406.00,615.00,824.00,1033.00,586.25,670.00,837.50,972.50,1106.25,938.75,1072.50,1340.00,1555.00,1770.00,1137.50,1300.00,1625.00,1885.00,2145.00,1365.00,1560.00,1950.00,2262.50,2573.75,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,79.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4635,H,5301,35.00,4.00,2.00,nan,nan,3400.00,nan,3.00,nan,nan,3540.00,32.00,133000.00,2.00,nan,Monterey CA,NaN,NaN,NaN,NaN,NaN,NaN,Monterey,427.50,488.75,615.00,824.00,103

# Part D1. Add dummy variables for vacant "for-rent," vacant "for-sale," and recently moved in

In [55]:
# create dummy variables for vacant for rent, vacant for sale, rented in the last year, and sold in the last year
units_df["vacant_for_rent"]=np.where(units_df['vacancy_status']==1,1,0)
units_df["vacant_for_sale"]=np.where(units_df['vacancy_status']==3,1,0)
units_df["moved_last_year"]=np.where(units_df['moved_in']==1,1,0)
units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,...,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_monthly_cost,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count,vacant_for_rent,vacant_for_sale,moved_last_year
0,H,6703,59.00,2.00,9.00,nan,nan,940.00,nan,3.00,nan,nan,990.00,25.00,47000.00,3.00,nan,Sacramento CA,NaN,NaN,NaN,NaN,NaN,NaN,Sacramento,400.00,457.50,615.00,824.00,1033.00,666.25,761.25,951.25,1103.75,1256.25,1066.25,1218.75,1522.50,1766.25,2010.00,1331.25,1522.50,1902.50,2207.50,2511.25,1597.50,1826.25,2282.50,2647.50,3012.50,...,0.00,0.00,0.00,59.00,0.00,0.00,0.00,0.00,59.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0
1,H,110,74.00,4.00,2.00,0.00,2600.00,nan,nan,1.00,nan,780000.00,nan,nan,130500.00,1.00,67.00,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN,Alameda,548.75,626.25,782.50,908.75,1033.75,913.75,1043.75,1303.75,1512.50,1721.25,1407.50,1608.75,2010.00,2332.50,2653.75,1705.00,1947.50,2435.00,2825.00,3213.75,2046.25,2337.50,2922.50,3390.00,3857.50,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,1
2,H,7901,111.00,2.00,6.00,nan,nan,nan,nan,nan,5.00,nan,nan,nan,nan,nan,nan,San Luis Obispo CA,NaN,NaN,NaN,NaN,NaN,NaN,San Luis Obispo,428.75,490.00,615.00,824.00,1033.00,715.00,817.50,1021.25,1185.00,1348.75,1143.75,1307.50,1633.75,1896.25,2157.50,1456.25,1663.75,2080.00,2412.50,2745.00,1747.50,1997.50,2496.25,2896.25,3295.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0
3,H,1906,63.00,3.00,2.00,0.00,nan,nan,nan,2.00,nan,175000.00,nan,nan,13530.00,7.00,14.00,Fresno CA,NaN,NaN,NaN,NaN,NaN,NaN,Fresno,315.00,406.00,615.00,824.00,988.75,525.00,600.00,748.75,868.75,988.75,838.75,958.75,1197.50,1390.00,1581.25,1048.75,1197.50,1497.50,1737.50,1976.25,1258.75,1437.50,1797.50,2085.00,2372.50,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0
4,H,6710,75.00,5.00,2.00,0.00,nan,nan,nan,2.00,nan,500000.00,nan,nan,104000.00,4.00,48.00,Sacramento CA,NaN,NaN,NaN,NaN,NaN,NaN,Sacramento,400.00,457.50,615.00,824.00,1033.00,666.25,761.25,951.25,1103.75,1256.25,1066.25,1218.75,1522.50,1766.25,2010.00,1331.25,1

In [56]:
#filter for only currently listed or recently moved into units
recent_mask = (units_df.vacant_for_rent==1)|(units_df.vacant_for_sale==1)|(units_df.moved_last_year==1)
recent_units_df = units_df[recent_mask]
recent_units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,...,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_monthly_cost,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count,vacant_for_rent,vacant_for_sale,moved_last_year
1,H,110,74.00,4.00,2.00,0.00,2600.00,nan,nan,1.00,nan,780000.00,nan,nan,130500.00,1.00,67.00,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN,Alameda,548.75,626.25,782.50,908.75,1033.75,913.75,1043.75,1303.75,1512.50,1721.25,1407.50,1608.75,2010.00,2332.50,2653.75,1705.00,1947.50,2435.00,2825.00,3213.75,2046.25,2337.50,2922.50,3390.00,3857.50,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,1
10,H,6703,105.00,1.00,5.00,nan,nan,750.00,nan,3.00,nan,nan,870.00,16.00,65000.00,1.00,nan,Sacramento CA,NaN,NaN,NaN,NaN,NaN,NaN,Sacramento,400.00,457.50,615.00,824.00,1033.00,666.25,761.25,951.25,1103.75,1256.25,1066.25,1218.75,1522.50,1766.25,2010.00,1331.25,1522.50,1902.50,2207.50,2511.25,1597.50,1826.25,2282.50,2647.50,3012.50,...,0.00,0.00,105.00,0.00,0.00,0.00,0.00,105.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,1
18,H,3730,57.00,2.00,9.00,nan,nan,2700.00,nan,3.00,nan,nan,2764.00,33.00,100000.00,1.00,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,473.75,541.25,676.25,824.00,1033.00,788.75,901.25,1126.25,1307.50,1487.50,1262.50,1442.50,1802.50,2091.25,2380.00,1133.75,1296.25,1620.00,1878.75,2138.75,1361.25,1555.00,1943.75,2255.00,2566.25,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,57.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,1
24,H,3732,160.00,0.00,9.00,nan,nan,1400.00,nan,3.00,nan,nan,1490.00,26.00,70000.00,1.00,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,473.75,541.25,676.25,824.00,1033.00,788.75,901.25,1126.25,1307.50,1487.50,1262.50,1442.50,1802.50,2091.25,2380.00,1133.75,1296.25,1620.00,1878.75,2138.75,1361.25,1555.00,1943.75,2255.00,2566.25,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,160.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,1
29,H,6501,70.00,2.00,4.00,200.00,3400.00,nan,nan,1.00,nan,1200000.00,nan,nan,230000.00,1.00,68.00,Riverside CA,NaN,NaN,NaN,NaN,NaN,NaN,Riverside,352.50,406.00,615.00,824.00,1033.00,586.25,670.00,837.50,972.50,1106.25,938.75,10

# Part D2. Aggregating at the PUMA level

In [57]:
# for all units
puma_sums_df = units_df.groupby("PUMA").sum()
puma_counts_df = puma_sums_df[["rent_ELI_studio_count","rent_ELI_1_br_count",
    "rent_ELI_2_br_count",
    "rent_ELI_3_br_count",
    "rent_ELI_4_br_count",
    "rent_VLI_studio_count",
    "rent_VLI_1_br_count",
    "rent_VLI_2_br_count",
    "rent_VLI_3_br_count",
    "rent_VLI_4_br_count",
    "rent_LI_studio_count",
    "rent_LI_1_br_count",
    "rent_LI_2_br_count",
    "rent_LI_3_br_count",
    "rent_LI_4_br_count",
    "rent_MI_studio_count",
    "rent_MI_1_br_count",
    "rent_MI_2_br_count",
    "rent_MI_3_br_count",
    "rent_MI_4_br_count",
    "rent_MoI_studio_count",
    "rent_MoI_1_br_count",
    "rent_MoI_2_br_count",
    "rent_MoI_3_br_count",
    "rent_MoI_4_br_count",
    "rent_HI_studio_count",
    "rent_HI_1_br_count",
    "rent_HI_2_br_count",
    "rent_HI_3_br_count",
    "rent_HI_4_br_count",
    "own_ELI_studio_count",
    "own_ELI_1_br_count",
    "own_ELI_2_br_count",
    "own_ELI_3_br_count",
    "own_ELI_4_br_count",
    "own_VLI_studio_count",
    "own_VLI_1_br_count",
    "own_VLI_2_br_count",
    "own_VLI_3_br_count",
    "own_VLI_4_br_count",
    "own_LI_studio_count",
    "own_LI_1_br_count",
    "own_LI_2_br_count",
    "own_LI_3_br_count",
    "own_LI_4_br_count",
    "own_MI_studio_count",
    "own_MI_1_br_count",
    "own_MI_2_br_count",
    "own_MI_3_br_count",
    "own_MI_4_br_count",
    "own_MoI_studio_count",
    "own_MoI_1_br_count",
    "own_MoI_2_br_count",
    "own_MoI_3_br_count",
    "own_MoI_4_br_count",
    "own_HI_studio_count",
    "own_HI_1_br_count",
    "own_HI_2_br_count",
    "own_HI_3_br_count",
    "own_HI_4_br_count"]].copy()
puma_counts_df

,rent_ELI_studio_count,rent_ELI_1_br_count,rent_ELI_2_br_count,rent_ELI_3_br_count,rent_ELI_4_br_count,rent_VLI_studio_count,rent_VLI_1_br_count,rent_VLI_2_br_count,rent_VLI_3_br_count,rent_VLI_4_br_count,rent_LI_studio_count,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count
PUMA,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
101,178.00,1009.00,571.00,66.00,216.00,1241.00,2742.00,3274.00,1040.00,588.00,2334.00,6465.00,7893.00,1894.00,622.00,2572.00,8504.00,9207.00,2042.00,701.00,2951.00,9449.00,10929.00,2385.00,701.00,211.00,1069.00,1313.00,285.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,236.00,93.00,0.00,0.00,320.00,393.00,460.00,0.00,0.00,381.00,541.00,460.00,0.00,0.00,449.00,924.00,510.00,0.00,0.00,403.00,592.00,158.00
102,1587.00,2743.00,2103.00,1342.00,104.00,4719.00,6837.00,8123.00,2759.00,576.00,7427.00,12004.00,13190.00,4122.00,888.00,8618.00,14968.00,14600.00,4629.00,1036.00,8693.00,17957.00,16052.00,5061.00,1401.00,1086.00,3527.00,1678.00,0.00,0.00,0.00,0.00,55.00,87.00,0.00,0.00,0.00,55.00,87.00,47.00,0.00,60.00,457.00,368.00,105.00,0.00,119.00,457.00,521.00,165.00,0.00,181.00,520.00,521.00,240.00,0.00,240.00,262.00,153.00,196.00
103,366.00,145.00,576.00,406.00,63.00,867.00,1555.00,2544.00,874.00,63.00,1224.00,4125.00,4856.00,1082.00,160.00,1911.00,5962.00,5181.00,1490.00,160.00,2067.00,6550.00,5761.00,2378.00,433.00,0.00,456.00,624.00,421.00,0.00,0.00,0.00,81.00,0.00,0.00,0.00,0.00,81.00,0.00,108.00,0.00,0.00,316.00,563.00,245.00,0.00,0.00,777.00,1042.00,495.00,0.00,171.00,842.00,1261.00,674.00,0.00,89.00,377.00,503.00,495.00
104,631.00,864.00,1738.00,554.00,0.00,1187.00,2879.00,7095.00,3234.00,200.00,2249.00,4382.00,11558.00,5238.00,507.00,2249.00,4605.00,12143.00,5668.00,661.00,2249.00,4871.00,12143.00,5719.00,661.00,66.00,0.00,0.00,111.00,0.00,0.00,0.00,150.00,84.00,0.00,0.00,0.00,195.00,144.00,0.00,0.00,0.00,534.00,337.00,0.00,0.00,0.00,623.00,337.00,392.00,0.00,61.00,623.00,394.00,392.00,0.00,98.00,133.00,0.00,0.00
105,327.00,1142.00,846.00,384.00,178.00,737.00,2230.00,2920.00,1798.00,290.00,1894.00,5152.00,9351.00,3759.00,290.00,2351.00,7395.00,11779.00,4651.00,599.00,2351.00,7928.00,12631.00,5383.00,695.00,448.00,398.00,521.00,203.00,0.00,0.00,0.00,0.00,63.00,103.00,0.00,0.00,101.00,194.00,103.00,0.00,0.00,159.00,456.00,501.00,0.00,0.00,220.00,691.00,1090.00,0.00,0.00,395.00,902.00,1179.00,0.00,0.00,654.00,95.00,374.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11103,232.00,798.00,706.00,755.00,829.00,232.00,2146.00,2500.00,1390.00,1252.00,563.00,5497.00,7685.00,5584.00,3387.00,563.00,5799.00,8214.00,5632.00,3655.00,679.00,6025.00,9167.00,5632.00,3655.00,0.00,0.00,67.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,110.00,366.00,0.00,0.00,0.00,257.00,500.00,0.00,0.00,0.00,617.00,500.00,0.00,0.00,0.00,770.00,604.00,0.00,0.00,0.00,105.00,393.00
11104,139.00,1022.00,320.00,324.00,205.00,498.00,1821.00,1992.00,810.00,339.0

In [58]:
#export it
puma_counts_df.to_csv("weighted_puma_counts.csv")

In [59]:
# for recent units
recent_puma_sums_df = recent_units_df.groupby("PUMA").sum()
recent_puma_counts_df = recent_puma_sums_df[["rent_ELI_studio_count","rent_ELI_1_br_count",
    "rent_ELI_2_br_count",
    "rent_ELI_3_br_count",
    "rent_ELI_4_br_count",
    "rent_VLI_studio_count",
    "rent_VLI_1_br_count",
    "rent_VLI_2_br_count",
    "rent_VLI_3_br_count",
    "rent_VLI_4_br_count",
    "rent_LI_studio_count",
    "rent_LI_1_br_count",
    "rent_LI_2_br_count",
    "rent_LI_3_br_count",
    "rent_LI_4_br_count",
    "rent_MI_studio_count",
    "rent_MI_1_br_count",
    "rent_MI_2_br_count",
    "rent_MI_3_br_count",
    "rent_MI_4_br_count",
    "rent_MoI_studio_count",
    "rent_MoI_1_br_count",
    "rent_MoI_2_br_count",
    "rent_MoI_3_br_count",
    "rent_MoI_4_br_count",
    "rent_HI_studio_count",
    "rent_HI_1_br_count",
    "rent_HI_2_br_count",
    "rent_HI_3_br_count",
    "rent_HI_4_br_count",
    "own_ELI_studio_count",
    "own_ELI_1_br_count",
    "own_ELI_2_br_count",
    "own_ELI_3_br_count",
    "own_ELI_4_br_count",
    "own_VLI_studio_count",
    "own_VLI_1_br_count",
    "own_VLI_2_br_count",
    "own_VLI_3_br_count",
    "own_VLI_4_br_count",
    "own_LI_studio_count",
    "own_LI_1_br_count",
    "own_LI_2_br_count",
    "own_LI_3_br_count",
    "own_LI_4_br_count",
    "own_MI_studio_count",
    "own_MI_1_br_count",
    "own_MI_2_br_count",
    "own_MI_3_br_count",
    "own_MI_4_br_count",
    "own_MoI_studio_count",
    "own_MoI_1_br_count",
    "own_MoI_2_br_count",
    "own_MoI_3_br_count",
    "own_MoI_4_br_count",
    "own_HI_studio_count",
    "own_HI_1_br_count",
    "own_HI_2_br_count",
    "own_HI_3_br_count",
    "own_HI_4_br_count"]].copy()
recent_puma_counts_df

,rent_ELI_studio_count,rent_ELI_1_br_count,rent_ELI_2_br_count,rent_ELI_3_br_count,rent_ELI_4_br_count,rent_VLI_studio_count,rent_VLI_1_br_count,rent_VLI_2_br_count,rent_VLI_3_br_count,rent_VLI_4_br_count,rent_LI_studio_count,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count
PUMA,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
101,0.00,136.00,115.00,0.00,0.00,165.00,373.00,701.00,97.00,110.00,766.00,947.00,1903.00,220.00,110.00,811.00,1129.00,2141.00,220.00,110.00,1014.00,1650.00,2831.00,255.00,110.00,51.00,644.00,339.00,60.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,92.00,0.00,0.00,0.00,0.00,0.00,0.00
102,269.00,89.00,92.00,44.00,0.00,516.00,142.00,1004.00,252.00,223.00,874.00,1221.00,1498.00,416.00,223.00,1085.00,2664.00,2038.00,614.00,371.00,1085.00,3646.00,2729.00,675.00,459.00,365.00,1949.00,696.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,59.00,0.00,103.00,0.00,0.00,59.00,63.00,103.00,75.00,0.00,0.00,0.00,0.00,0.00
103,0.00,0.00,0.00,0.00,0.00,57.00,106.00,164.00,94.00,0.00,198.00,345.00,213.00,156.00,0.00,756.00,1331.00,213.00,194.00,0.00,792.00,1410.00,340.00,194.00,128.00,0.00,251.00,419.00,56.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,63.00,0.00,0.00,0.00,68.00,0.00
104,310.00,146.00,185.00,0.00,0.00,310.00,316.00,515.00,240.00,0.00,696.00,587.00,1154.00,430.00,0.00,696.00,751.00,1223.00,789.00,154.00,696.00,953.00,1223.00,789.00,154.00,0.00,0.00,0.00,111.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,34.00,0.00,0.00,0.00,0.00,34.00,0.00,0.00,0.00,0.00,34.00,0.00,0.00,0.00,0.00,0.00,0.00
105,0.00,115.00,0.00,53.00,0.00,159.00,115.00,212.00,168.00,0.00,809.00,115.00,812.00,168.00,0.00,1154.00,1177.00,1724.00,272.00,53.00,1154.00,1472.00,2075.00,473.00,53.00,184.00,135.00,187.00,130.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,68.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11103,90.00,0.00,0.00,0.00,275.00,90.00,326.00,0.00,196.00,275.00,306.00,1737.00,753.00,836.00,665.00,306.00,1836.00,1133.00,884.00,933.00,422.00,1913.00,1538.00,884.00,933.00,0.00,0.00,67.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
11104,0.00,355.00,107.00,0.00,0.00,103.00,803.00,510.00,0.00,0.00,131.00,1338.00,2075.00,417.00,33.00,131.00,1684.00,2294.00,550.00,113.00,131.00,2164.00,2437.00,793.00,183.00,110.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,60.00
11105,94.00,0.00,287.00,35.00,0.

In [60]:
#export it
recent_puma_counts_df.to_csv("weighted_recent_puma_counts.csv")